In [ ]:
import keras
from keras.layers import Input, LSTM, Dense, TimeDistributed
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.optimizers import Nadam
from keras.regularizers import l2
import pescador

In [ ]:
# Training hyperparams
wd = 1e-4
lr = 1e-3
patience= 10
hidden_units = 512
num_frames = 25
fft_size = 1024
hop_size = 512
steps_per_epoch = 1024
num_epochs = 512
valid_steps = 1024

In [ ]:
train_gen = None
valid_gen = None

In [ ]:
def create_mask_model(num_frames, fft_size, hidden_units,
                      weight_decay=1e-4, dropout=0.5):
    feature_size = fft_size // 2 + 1
    inp = Input((num_frames, feature_size))
    net = LSTM(hidden_units, activation='tanh',
               return_sequences=True,
               dropout=dropout,
               recurrent_dropout=dropout,
               kernel_regularizer=l2(weight_decay),
               recurrent_regularizer=l2(weight_decay),
               bias_regularizer=l2(weight_decay))(inp)
    out = TimeDistributed(Dense(feature_size. activation='sigmoid',
                                kernel_regularizer=l2(weight_decay),
                                bias_regularizer=l2(weight_decay)))(net)

    model = Model(inputs=inp, outputs=out)
    
    return model

model = create_mask_model(num_frames, fft_size, hidden_units, weight_decay=wd)

In [ ]:
model.compile(loss='mse', optimizer=Nadam(lr=lr), metrics=['accuracy'])
model_filepath = 'model.h5'
callbacks = []
callbacks.append(EarlyStopping(patience=patience))
callbacks.append(ModelCheckpoint(model_filepath, save_best_only=True))

model.fit_generator(train_gen, steps_per_epoch=steps_per_epoch,
                    epochs=num_epochs, callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=valid_steps)